In [ ]:
import numpy as np
import pandas as pd
import glob

In [ ]:
def curveFit(f, pinkNoise, ThermalNoise, measuredVal):
    return pinkNoise/f + ThermalNoise - measuredVal

In [ ]:
def readData(fileName: str):
    data = pd.read_csv(fileName)
    return data

In [ ]:
dataFiles = glob.glob("data/*")
